In [173]:
import agate

In [174]:
exonerations = agate.Table.from_csv('exonerations-20150828.csv')

#tester = agate.TypeTester(force={
    #'false_evidence': agate.Boolean()
    #'perjury': agate.Boolean()
    #'official_misconduct': agate.Boolean()
#})

#exonerations = agate.Table.from_csv('exonerations-20150828.csv', column_types=tester)`

In [175]:
#tester = agate.TypeTester(limit=100)

#exonerations = agate.Table.from_csv('exonerations-20150828.csv', tester)

In [176]:
print exonerations.columns['last_name']


<agate.Column: (u'Abbitt', u'Abdal', u'Abernathy', u'Acero', u'Adams', ...)>


In [177]:
print exonerations.columns[0]

<agate.Column: (u'Abbitt', u'Abdal', u'Abernathy', u'Acero', u'Adams', ...)>


In [178]:
firstrow = exonerations.rows[0]

print firstrow


<agate.Row: (u'Abbitt', u'Joseph Lamont', Decimal('31'), u'Black', u'NC', ...)>


In [179]:
print firstrow['dna']

True


In [180]:
num_false_confessions = exonerations.aggregate(agate.Count('false_confession', True))

print(num_false_confessions)

211


In [181]:
median_age = exonerations.aggregate(agate.Median('age'))

print(median_age)


26


In [182]:
num_without_age = exonerations.aggregate(agate.Count('age', None))

print(num_without_age)

9


In [198]:
mo = exonerations.aggregate(agate.Count("state", "MO"))
print mo

34


In [184]:
with_age = exonerations.where(lambda row: row['age'] is not None)


In [185]:
old = len(exonerations.rows)
new = len(with_age.rows)

print(old - new)

9


In [186]:
median_age = with_age.aggregate(agate.Median('age'))

print(median_age)

26


In [187]:
with_years_in_prison = exonerations.compute([
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

median_years = with_years_in_prison.aggregate(agate.Median('years_in_prison'))

print(median_years)

8


In [188]:
full_names = exonerations.compute([
    ('full_name', agate.Formula(agate.Text(), lambda row: '%(first_name)s %(last_name)s' % row))
])

In [189]:
with_computations = exonerations.compute([
    ('full_name', agate.Formula(agate.Text(), lambda row: '%(first_name)s %(last_name)s' % row)),
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])


In [190]:
sorted_by_age = exonerations.order_by('age')

In [191]:
youngest_ten = sorted_by_age.limit(10)

In [192]:
youngest_ten.print_table(max_columns=7)

|------------+------------+-----+-----------+-------+---------+---------+------|
|  last_name | first_name | age | race      | state | tags    | crime   | ...  |
|------------+------------+-----+-----------+-------+---------+---------+------|
|  Murray    | Lacresha   |  11 | Black     | TX    | CV, F   | Murder  | ...  |
|  Adams     | Johnathan  |  12 | Caucasian | GA    | CV, P   | Murder  | ...  |
|  Harris    | Anthony    |  12 | Black     | OH    | CV      | Murder  | ...  |
|  Edmonds   | Tyler      |  13 | Caucasian | MS    |         | Murder  | ...  |
|  Handley   | Zachary    |  13 | Caucasian | PA    | A, CV   | Arson   | ...  |
|  Jimenez   | Thaddeus   |  13 | Hispanic  | IL    |         | Murder  | ...  |
|  Pacek     | Jerry      |  13 | Caucasian | PA    |         | Murder  | ...  |
|  Barr      | Jonathan   |  14 | Black     | IL    | CDC, CV | Murder  | ...  |
|  Brim      | Dominique  |  14 | Black     | MI    | F       | Assault | ...  |
|  Brown     | Timothy    | 

In [193]:
binned_ages = exonerations.bins('age', 10, 0, 100)
binned_ages.print_bars('age', 'Count', width=80)

age       Count
[10 - 20)   307 ▓░░░░░░░░░░░░░░░░░░░░░░░░                                       
[20 - 30)   718 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░      
[30 - 40)   377 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                                 
[40 - 50)   176 ▓░░░░░░░░░░░░░░                                                 
[50 - 60)    53 ▓░░░░                                                           
[60 - 70)    10 ▓░                                                              
[80 - 90)     1 ▓                                                               
None          9 ▓░                                                              
                +---------------+---------------+--------------+---------------+
                0              200             400            600            800


In [194]:
by_state = exonerations.group_by('state')

In [195]:
state_totals = by_state.aggregate([
    ('count', agate.Count())
])

sorted_totals = state_totals.order_by('count', reverse=True)

sorted_totals.print_table(max_rows=5)

|--------+--------|
|  state | count  |
|--------+--------|
|  TX    |   212  |
|  NY    |   202  |
|  CA    |   154  |
|  IL    |   153  |
|  MI    |    60  |
|  ...   |   ...  |
|--------+--------|


In [196]:
with_years_in_prison = exonerations.compute([
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

state_totals = with_years_in_prison.group_by('state')

medians = state_totals.aggregate([
    ('count', agate.Count()),
    ('median_years_in_prison', agate.Median('years_in_prison'))
])

sorted_medians = medians.order_by('median_years_in_prison', reverse=True)

sorted_medians.print_table(max_rows=5)

|--------+-------+-------------------------|
|  state | count | median_years_in_prison  |
|--------+-------+-------------------------|
|  DC    |    15 |                     27  |
|  NE    |     9 |                     20  |
|  ID    |     2 |                     19  |
|  VT    |     1 |                     18  |
|  LA    |    45 |                     16  |
|  ...   |   ... |                    ...  |
|--------+-------+-------------------------|


In [197]:
# Filters rows without age data
only_with_age = with_years_in_prison.where(
    lambda r: r['age'] is not None
)

# Group by race
race_groups = only_with_age.group_by('race')

# Sub-group by age cohorts (20s, 30s, etc.)
race_and_age_groups = race_groups.group_by(
    lambda r: '%i0s' % (r['age'] // 10),
    key_name='age_group'
)

# Aggregate medians for each group
medians = race_and_age_groups.aggregate([
    ('count', agate.Count()),
    ('median_years_in_prison', agate.Median('years_in_prison'))
])

# Sort the results
sorted_groups = medians.order_by('median_years_in_prison', reverse=True)

# Print out the results
sorted_groups.print_table(max_rows=10)

|------------------+-----------+-------+-------------------------|
|  race            | age_group | count | median_years_in_prison  |
|------------------+-----------+-------+-------------------------|
|  Native American | 20s       |     2 |                   21.5  |
|                  | 20s       |     1 |                   19.0  |
|  Native American | 10s       |     2 |                   15.0  |
|  Native American | 30s       |     2 |                   14.5  |
|  Black           | 10s       |   188 |                   14.0  |
|  Black           | 20s       |   358 |                   13.0  |
|  Asian           | 20s       |     4 |                   12.0  |
|  Black           | 30s       |   156 |                   10.0  |
|  Caucasian       | 10s       |    76 |                    8.0  |
|  Caucasian       | 20s       |   255 |                    8.0  |
|  ...             | ...       |   ... |                    ...  |
|------------------+-----------+-------+----------------------